In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc
import json
import csv
import cv2
import scipy
import os

In [7]:
num_classes = 4
epochs = 5
# BASE_PATH = '/home/ec2-user/cell_classifier/'
BASE_DIR = '../'
batch_size = 32

In [3]:
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(240, 320, 3), output_shape=(240, 320, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(240, 320, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

    return model

In [4]:
def get_data(folder):
    X = []
    y = []

    for wbc_type in os.listdir(folder):
        if not wbc_type.startswith('.'):
            for image_filename in os.listdir(folder + wbc_type):
                img_file = cv2.imread(folder + wbc_type + '/' + image_filename)
                if img_file is not None:
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(wbc_type)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [8]:
X_train, y_train = get_data(BASE_DIR + 'images/TRAIN/')
X_test, y_test = get_data(BASE_DIR + 'images/TEST/')

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoded_y_test = encoder.transform(y_test)

y_train = np_utils.to_categorical(encoded_y_train)
y_test = np_utils.to_categorical(encoded_y_test)

In [9]:
print(y_train)

[[ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]]


In [10]:
model = get_model()

# fits the model on batches
model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=epochs,
    shuffle=True,
    batch_size=batch_size)

Train on 15873 samples, validate on 3969 samples
Epoch 1/5
15873/15873 [==============================] - 185s - loss: 1.1178 - acc: 0.5336 - val_loss: 3.3576 - val_acc: 0.0000e+00
Epoch 2/5
15873/15873 [==============================] - 181s - loss: 0.7182 - acc: 0.7517 - val_loss: 1.8926 - val_acc: 0.0000e+00
Epoch 3/5
15873/15873 [==============================] - 181s - loss: 0.4785 - acc: 0.8338 - val_loss: 3.2436 - val_acc: 0.0000e+00
Epoch 4/5
15873/15873 [==============================] - 181s - loss: 0.3626 - acc: 0.8797 - val_loss: 4.4355 - val_acc: 0.0141
Epoch 5/5
15873/15873 [==============================] - 181s - loss: 0.2852 - acc: 0.9100 - val_loss: 2.0186 - val_acc: 0.2960


In [13]:
from sklearn.metrics import accuracy_score

print('Predicting on test data')
y_pred = np.rint(model.predict(X_test))

# saving weights and stats
model.save_weights(os.path.join(os.getcwd(),'balanced_model.h5'))
stats = {}
stats['accuracy_score'] = accuracy_score(y_test, y_pred)
stats_filename = os.path.join(os.getcwd(),'stats.json')
with open(stats_filename, 'wb') as f:
    f.write(json.dumps(stats))
print stats

Predicting on test data
{'accuracy_score': 0.64977885002010449}
